在实际工作中，FM和MF哪个应用的更多?
FM:因子分解机，是一种基于矩阵分解思想的机器学习算法，是为了结局大规模稀疏矩阵中特征组合的问题。
MF：矩阵分解，将用户与物品嵌入到一个共同的低维空间内。
MF是FM的一个特例。
FM降低时间复杂度为k*n。

FFM与FM有哪些区别？
FFM在FM的基础上引入了field的概念，每个特征的隐向量不在只有一个，而是针对每个field学习一个独立的隐向量，防止互相影响。

DeepFM相比于FM解决了哪些问题，原理是怎样的？
DeepFM = FM+DNN ,可以考虑低阶，又能考虑到高阶的特征工程。因子分解机FM提取低阶的特征，神经网络DNN提取高阶特征。


Surprise工具中的baseline算法原理是怎样的？BaselineOnly和KNNBaseline有什么区别？
baseline算法：基于统计的基准预测线打分，bui = μ+bu+bi
使用ALS进行优化，step1：固定bu，优化bi， step2：固定bi，优化bu。


#action使用libfm工具对movielens进行评分预测，采用SGD优化算法
libFM -task r -train ratings.dat.libfm -test ratings.dat.libfm -dim '1,1,8' -method sgd -learn_rate 0.01 -regular ’0,0,0.01’ -out 

In [1]:
#action2使用DeepFM对movielens进行评分预测
import pandas as pd
from deepctr.models import DeepFM
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from deepctr.1 import SparseFeat,get_feature_names

In [2]:
data = pd.read_csv(r"C:\Users\zhanghangbo\Desktop\数据挖掘算法\第五周\L5\deepfm\movielens_sample.txt")
data.shape
sparse_features = ["movie_id","user_id","gender","age","occupation","zip"]
target = ["rating"]
#对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature]= lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
print(fixlen_feature_columns)
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[SparseFeat(name='movie_id', vocabulary_size=187, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x000001B4E763F278>, embedding_name='movie_id', group_name='default_group', trainable=True), SparseFeat(name='user_id', vocabulary_size=193, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x000001B4E7661EF0>, embedding_name='user_id', group_name='default_group', trainable=True), SparseFeat(name='gender', vocabulary_size=2, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x000001B4E7661F28>, embedding_name='gender', group_name='default_group', trainable=True), SparseFeat(name='age', vocabulary_size=7, embedding_dim=4, use_hash

In [3]:
# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 128 samples, validate on 32 samples
128/128 [==============================] - 0s 2ms/sample - loss: 14.3671 - mean_squared_error: 14.3671 - val_loss: 13.7776 - val_mean_squared_error: 13.7776
test RMSE 3.678314831549904


In [4]:
#movielens数据集
data = pd.read_csv(r"C:\Users\zhanghangbo\Desktop\数据挖掘算法\第五周\L5\libfm\ratings.csv")
data.head(5)
sparse_features = ["userId","movieId"]
target = ["rating"]
#对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature]= lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
print(fixlen_feature_columns)
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

[SparseFeat(name='userId', vocabulary_size=7120, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x000001B4E7661AC8>, embedding_name='userId', group_name='default_group', trainable=True), SparseFeat(name='movieId', vocabulary_size=14026, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x000001B4E7661A90>, embedding_name='movieId', group_name='default_group', trainable=True)]


In [5]:
# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

Train on 671088 samples, validate on 167772 samples
671088/671088 [==============================] - 4s 6us/sample - loss: 0.9522 - mean_squared_error: 0.9514 - val_loss: 0.7436 - val_mean_squared_error: 0.7420
test RMSE 0.8616843969807043


In [25]:
#action使用基于邻域的协同过滤（KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline中的任意一种）
#对MovieLens数据集进行协同过滤，采用k折交叉验证(k=3)，输出每次计算的RMSE, MAE

In [6]:
from surprise import KNNBasic
from surprise import Dataset,Reader
from surprise.accuracy  import rmse
from surprise.model_selection import KFold

In [7]:
#数据读取
reader = Reader(line_format="user item rating timestamp",sep=",",skip_lines=1)
data = Dataset.load_from_file(r"C:\Users\zhanghangbo\Desktop\数据挖掘算法\第五周\L5\libfm\ratings.csv",reader=reader)
train_set =data.build_full_trainset()
#itemcf计算得分
algo = KNNBasic(k=50,sim_options={"user_based":True})
kf = KFold(n_splits=3)
for trainset,testet in kf.split(data):
    algo.fit(trainset)
    pred = algo.test(testet)
    rmse(pred,verbose=True)
uid = str(196)
lid = str(302)
predction = algo.predict(uid,lid) 


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9023
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9025
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9053
